# Singular Value Decomposition & Least-Squares Solution
## SVD
- SVD는 A가 어느 행렬이건 가능하다!
- $A = U \sum {V}^T / A = U \sum {V}^*$
        - U, V: orthogonal matrix / unitary matrix(congugate & Transpose) $\sum$: singular value로 이뤄진 matrix
- **linalg.svd**
    - Lapack: gesdd
- U, s, **VT** = linalg.svd(A, compute_uv=True)
    - compute_uv=False로 설정을 하면 s만 반환
    - U: m x m
    - s: 1D array min(m,n) singular values (큰 것부터 작은 순으로 0도 포함될 수 있음)
    - VT: n x n
    - A: m x n
- A = U @ linalg.diagsvd(s, A.shap[0], A.shape[1]) @ VT

## Reduced SVD
- $A = \begin{bmatrix}
        {U}_r & {U}_{m-r} \\
        \end{bmatrix} \begin{bmatrix}
                       D & 0 \\
                       0 & 0 \\
                       \end{bmatrix} \begin{bmatrix}
                                      {V}_r^T \\
                                      {{V}_{n-r}}^T \\
                                      \end{bmatrix} = {U}_r D {V}_r^T$
    - D: nonzero singular values로 이뤄진 대각 행렬
    - r: rank A
- r = s.shape[0] - sum(np.allclose(lx, 0) for lx in s)
- r = s.shape[0] - sum(lx < 1.e-10 for lx in s)
    - 1e-8 이하는 0으로 취급함을 유의
- U[:, :r] @ np.diag(s[:r]) @ VT[:r, :]
- U[:, :r] * s[:r] @ VT[:r, :]

## Truncated SVD
- $A = {U}_r D {V}_r^T$
- $D = \begin{bmatrix}
       {\sigma}_1 & \, & \, \\
       \, & \ddots & \, \\
       \, & \, & {\sigma}_r \\
       \end{bmatrix}$
- U[:, :t] * s[:t] @ VT[:t, :]
- 상대적으로 ${\sigma}_{max}$대비 작은 $\sigma$들을 무시하고 싶다면?
    - t = sum(lx > 1.e-3 * s[0] for lx in s)
    - 만약 ${\sigma}_{max} = 1000$이면 1이하 $\sigma$들은 무시

In [ ]:
import numpy as np
from scipy import linalg
import timeit

In [9]:
A1 = np.array([[1,-1],[-2,2],[2,-2]])
U, s, VT = linalg.svd(A1)
print(U, s, VT)
# A1 reconstruct
re_A1 = U @ linalg.diagsvd(s, A1.shape[0], A1.shape[1]) @ VT
print(np.allclose(A1, re_A1))

A2 = np.array([[4,11,14],[8,7,-2]])
U, s, VT = linalg.svd(A2)
# A2 reconstruct
re_A2 = U @ linalg.diagsvd(s, A2.shape[0], A2.shape[1]) @ VT
print(np.allclose(A2, re_A2))

# random 10 x 10 matrix
A3 = np.random.rand(10, 10)
U, s, VT = linalg.svd(A3)
print(A3)
re_A3 = U @linalg.diagsvd(s, A3.shape[0], A3.shape[1]) @ VT
print(np.allclose(A3, re_A3))

[[-0.33333333  0.66666667 -0.66666667]
 [ 0.66666667  0.66666667  0.33333333]
 [-0.66666667  0.33333333  0.66666667]] [4.24264069 0.        ] [[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]
True
True
[[0.28671513 0.70415629 0.80344922 0.43402074 0.34661058 0.07957681
  0.90301752 0.72616605 0.46646759 0.79829405]
 [0.31644428 0.61202421 0.42529984 0.42097876 0.53266841 0.67200364
  0.06747865 0.39600985 0.76623445 0.14314985]
 [0.74488371 0.19524672 0.1252012  0.33907322 0.45441441 0.00605966
  0.21935731 0.44847481 0.07667982 0.73697345]
 [0.75317679 0.98207812 0.69414177 0.65911776 0.75377419 0.82500336
  0.64890876 0.86436871 0.95329543 0.849908  ]
 [0.31511725 0.48134781 0.03029548 0.15065236 0.72524746 0.0717256
  0.42169358 0.85978346 0.17208025 0.82563469]
 [0.42491831 0.65162933 0.21061294 0.45419046 0.47812459 0.15083459
  0.21507781 0.7737638  0.3108297  0.70711363]
 [0.19177337 0.79035971 0.28602671 0.57773362 0.1731209  0.71178568
  0.32109224 0.45057258 0.43953124 0

## Column space 와 Null space
- ColA = linalg.orth(A, rcond=None)
- NullA = linalg.null_space(A, rcond=None)

## Pseudoinverse (Moore-Penrose inverse)
- SVD: $A = U \sum {V}^T$ 
- reduced SVD: $A = {U}_r D {V}_r^T$
- pseudoinverse: ${A}^+ = {V}_r {D}^{-1} {U}_r^T$
- Ax = b , ${A}^+b$: least-squares solution
- **linalg.pinv**
- pinv_A = linalg.pinv(A)
- 어떤 행렬이건 SVD가 가능하다 $\to$ Pseudoinverse도 가능하다.

#### 상대적으로 작은 singular value들을 무시하고 싶을때
- pinv_A = linalg.pinv(A, **rcond=val**)
    - val * ${\sigma}_{max}$ 이하의 $\sigma$는 무시
- linalg.orth, linalg.null_space의 rcond도 같은 의미의 옵션!

## Least-Squares Solution
- **linalg.lstsq**
    - Lapack: gelsd
- x_hat, res, rank, s = linalg.lstsq(A,b, cond=None)
    - res: $\|b - A\hat{x}\|$: rank A < n (rank deficient: 비어 있는 데이터가 많다) or m < n (undetermined: 데이터가 부족하다): 주어진 데이터로 b를 근사하기에는 부족하다. **정상이면 비어있는 값** 
- pseudoinverse: ${A}^+ = {V}_r{D}^{-1}{U}_r^T$, ${A}^+b$:least-squares solution



In [10]:
# 연습 문제
## Practice 1.
# 1. 20 x 10 random 행령 만들기
A = np.random.rand(20, 10)

# 2. SVD 해보기
U, s, VT = linalg.svd(A)

# 3. Col A, Null A 살펴보기
ColA = linalg.orth(A)
NullA = linalg.null_space(A)

# 4. 20 사이즈의 random 벡터 만들기(b)
b = np.random.rand(20,)

# 5. Ax = b의 least-square 해 구하기
x_hat, res, rank, s = linalg.lstsq(A, b, cond=None)

# 6. A@x_hat과 b 비교
print(np.allclose(A@x_hat, b))


False
